# Using Torchserve on SageMaker

PyTorch Torchserve (**"PTS"**) is a native model server for PyTorch models. PTS was developed in collaboration between Meta and AWS to provide a production-ready model server for the PyTorch ecosystem. It allows you to serve and manage multiple models and serve requests via REST or gRPC endpoints. PTS supports serving TorchScripted models for better inference performance. It also comes with utilities to collect logs and metrics and optimization tweaks. SageMaker supports PTS as part of PyTorch [inference containers](https://github.com/aws/deep-learning-containers/tree/master/pytorch/inference/docker).

In this example we will use the Distilbert model that has been trained on the Q&A NLP task from HuggingFace Models.

### Preparing Model Artifacts

In our example we will our model using the `PyTorchModel` class from the Python SageMaker SDK. In this case, your model archive needs to provide only the necessary model artifacts (for example, model weights, lookups, tokenizers, and so on). As part of the `PyTorchModel` object configuration, you will provide your inference code and other dependencies, and SageMaker will automatically package it for PTS.

1. We start by fetching model artifacts and preparing model archive:

In [ ]:
! mkdir distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/pytorch_model.bin -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer_config.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/vocab.txt -P distilbert-base-uncased-distilled-squada
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/config.json -P distilbert-base-uncased-distilled-squad
!tar -C "$PWD" -czf distilbert-base-uncased-distilled-squad.tar.gz  distilbert-base-uncased-distilled-squad/

2. Then, we need to upload the model archive to Amazon S3. For this we import use SageMaker `Session` object:

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import os

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = 'torchserve'
s3_path = 's3://{}/{}'.format(bucket, prefix)


model_data = sagemaker_session.upload_data('distilbert-base-uncased-distilled-squad.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))

### Preparing Inference Script

SageMaker requires you to provide some code to load the model and run predictions so that you can preprocess incoming inference requests and post-process the response. To perform these operations, you need to implement the `model_fn()`, `predict_fn()`, `input_fn()`, and `output_fn()` methods. 

Run the cell below to review implementations of these methods:

In [ ]:
! pygmentize 2_src/pipeline_predictor.py

## Deploying Model

Deploying the model on PTS using the SageMaker SDK is straightforward. 

1. We first prepare `PyTorchModel` object. To configure PTS, we can use the `env` dictionary to set the appropriate environment variables in the serving container. Note that here, we explicitly reference the inference code via the `entry_point` parameter.

In [ ]:
from sagemaker.pytorch import PyTorchModel

env = {
    "SAGEMAKER_TS_BATCH_SIZE": "2",
    "SAGEMAKER_TS_MAX_BATCH_DELAY": "1000",
    "SAGEMAKER_TS_RESPONSE_TIMEOUT" : "120",
    "SAGEMAKER_TS_MIN_WORKERS" : "1",
    "SAGEMAKER_TS_MAX_WORKERS" : "2"
    }

model = PyTorchModel(model_data=model_data,
                   role=role, 
                   entry_point='pipeline_predictor.py',
                   source_dir='2_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_session)

2. Next, we define the endpoint configuration and supported serializers and deserializers for the request/response pair and deploy endpoint.

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


remote_predictor = model.deploy(initial_instance_count=1, instance_type="ml.g4dn.4xlarge", serializer=JSONSerializer(), deserializer=JSONDeserializer())

### Testing Deployed Model

Let's test our model. We prepare a question and its context in the cell below:

In [1]:
context = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question = "What kind of forest is Amazon?"
data = {"question":question, "context":context}

Then, we send prepared inference payload to endpoint.

In [ ]:
remote_predictor.predict(data)

### Resource Cleanup

Execute cell below to delete endpoints and model artifact:

In [ ]:
remote_predictor.delete_endpoint(delete_endpoint_config=True)
model.delete_model()